## Python TTS Service
conda env list
conda activate penv312
conda install fastapi uvicorn gtts python-multipart
conda install nest_asyncio

 Jupyter Notebook에서 실행하시려면 nest_asyncio를 사용하고, 
 일반적인 Python 파일(.py)로 저장해서 실행하실 경우에는 nest_asyncio 관련 부분은 제거하셔도 됩니다.
실행 후에는 http://localhost:8000/docs에서 FastAPI의 자동 생성된 문서를 확인하실 수 있습니다.

In [ ]:
from fastapi import FastAPI, HTTPException
from starlette.background import BackgroundTask  # BackgroundTask import 위치 변경
from pydantic import BaseModel
from gtts import gTTS
import tempfile
import os
from fastapi.responses import FileResponse
from fastapi.middleware.cors import CORSMiddleware
import nest_asyncio
import uvicorn

# nest_asyncio 적용
nest_asyncio.apply()

app = FastAPI()

# CORS 설정
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

class TTSRequest(BaseModel):
    text: str

@app.post("/tts")
async def text_to_speech(request: TTSRequest):
    try:
        # Create a temporary file to store the audio
        with tempfile.NamedTemporaryFile(delete=False, suffix='.mp3') as tmp_file:
            # Convert text to speech
            tts = gTTS(text=request.text, lang='ko')
            tts.save(tmp_file.name)
            
            # Return the audio file
            return FileResponse(
                tmp_file.name,
                media_type='audio/mpeg',
                filename='speech.mp3',
                background=BackgroundTask(lambda: os.unlink(tmp_file.name))
            )
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

if __name__ == "__main__":
    uvicorn.run(app, host="0.0.0.0", port=8000)

INFO:     Started server process [36508]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:57613 - "GET /tts HTTP/1.1" 405 Method Not Allowed
INFO:     127.0.0.1:57613 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:54083 - "POST /tts HTTP/1.1" 200 OK
INFO:     127.0.0.1:57286 - "POST /tts HTTP/1.1" 200 OK
INFO:     127.0.0.1:55008 - "POST /tts HTTP/1.1" 200 OK
INFO:     127.0.0.1:59124 - "POST /tts HTTP/1.1" 200 OK
